In [1]:
import numpy as np
import pandas as pd
import requests
import re
import json
from os import walk
from multiprocessing.pool import ThreadPool

In [2]:
URL = "https://s3.amazonaws.com/irs-form-990/index_2015.json"

response = requests.get(URL)
with open("index_2015.json", 'wb') as f:
    f.write(response.content)

In [3]:
with open("index_2015.json") as f:
    data = json.load(f)
    data = data[list(data.keys())[0]]

In [4]:
df = pd.DataFrame.from_dict(data)
df.tail(5)

,EIN,TaxPeriod,DLN,FormType,URL,OrganizationName,SubmittedOn,ObjectId,LastUpdated
261029,261916362,201412,93493274003245,990,https://s3.amazonaws.com/irs-form-990/20154274...,UNITED STATES CUTANEOUS LYMPHOMA CONSORTIUM,2015-10-14,201542749349300324,2016-04-29T13:40:20
261030,391531523,201412,93493274003395,990,https://s3.amazonaws.com/irs-form-990/20154274...,FRIENDS OF BEAVER CREEK RESERVE,2015-10-14,201542749349300339,2016-04-29T13:40:20
261031,237429891,201412,93493274004095,990,https://s3.amazonaws.com/irs-form-990/20154274...,SPOJNIA CREDIT UNION,2015-10-08,201542749349300409,2016-04-29T13:40:20
261032,330457856,201506,93493274004195,990,https://s3.amazonaws.com/irs-form-990/20154274...,CHILD ABUSE INTERVENTION FUND,2015-10-14,201542749349300419,2016-04-29T13:40:20
261033,741235398,201412,93493274004345,990,https://s3.amazonaws.com/irs-form-990/20154274...,GOOD SAMARITAN FOUNDATION OF TEXAS INC,2015-10-14,201542749349300434,2016-04-29T13:40:20


In [5]:
df.shape

(261034, 9)

In [6]:
def download_url(url):
    file_name_start = url.rfind('/') + 1
    file_name = url[file_name_start:]
    output_dir = "/home/meso/git_repo/exploring_990/data_2015/"
    file_name = output_dir + file_name
    
    r = requests.get(url, stream=True)
    if r.status_code == requests.codes.ok:
        with open(file_name, 'wb') as f:
            f.write(r.content)
    return url

urls = []
for i in range(df.shape[0]): 
    urls.append(data[i]['URL'])
    
results = ThreadPool(100).imap_unordered(download_url, urls)

In [7]:
f = []
for (dirpath, dirnames, filenames) in walk('/home/meso/git_repo/exploring_990/data_2015/'):
    f.extend(filenames)
    break

In [8]:
u = list(df['URL'])
len(u)

261034

In [9]:
file_names_list = []

for i in range(len(u)):
    file_loc = u[i].rfind("/") + 1
    file_name = u[i][file_loc:]
    file_names_list.append(file_name)

In [10]:
print(len(file_names_list), str(":"), len(f))

261034 : 260885


In [11]:
diff = list(set(file_names_list) - set(f))
len(diff)

149

In [12]:
file_loc = u[i].rfind("/") + 1
url_pre = u[0][:file_loc]

In [13]:
urls = []
for i in range(len(diff)):
    url = url_pre + diff[i]
    urls.append(url)

In [14]:
for url in urls:
    download_url(url)

In [15]:
f = []
for (dirpath, dirnames, filenames) in walk('/home/meso/git_repo/exploring_990/data_2015/'):
    f.extend(filenames)
    break

In [16]:
len(f)

261034

In [17]:
diff = list(set(file_names_list) - set(f))
len(diff)

0

In [18]:
def list_duplicates(seq):
    seen = set()
    seen_add = seen.add
    return [idx for idx, item in enumerate(seq) if item in seen or seen_add(item)]

In [19]:
print(list_duplicates(file_names_list))

[]


In [21]:
dup_df = df.iloc[list_duplicates(file_names_list)]

In [22]:
dup_ein = pd.DataFrame(dup_df['EIN'])

In [23]:
dup_ein

,EIN


In [24]:
duplicates = pd.DataFrame([])

for dup in range(dup_ein.shape[0]):
    ein = dup_ein.iloc[dup]['EIN']
    duplicates = duplicates.append(df[df['EIN'] == ein])

In [25]:
duplicates

""
